In [1]:
using XLSX
using JuMP
using SCIP
using GLPK
using Distributions
using StochasticPrograms

arquivo = XLSX.readxlsx("vjulia.xlsx");
W = arquivo["vento1!DI2:KT25"];# Wind generation forecast in scenario s period h[MW].
arquivo1 = XLSX.readxlsx("pjulia.xlsx");
P = arquivo1["preco!DI2:KT25"];# Expected market price in scenario period [R$/MWh].
tempo = [i for i in 1:24];

## Wind Farm Model

In [2]:
#20/06

w = 1 # Penalization
nCenarios = 1000;
a = [maximum(W[i,:]) for i in 1:24];
b = [minimum(W[i,:]) for i in 1:24];
c = [maximum(P[i,:]) for i in 1:24];
d = [minimum(P[i,:]) for i in 1:24];
Dist_a = [Uniform(b[i], a[i]) for i in 1:24]  ;            
Dist_b = [Uniform(d[i], c[i]) for i in 1:24];
M = zeros(Float64, 24,nCenarios);
N = zeros(Float64, 24,nCenarios);
for i in 1:24 
 Ma = rand(Dist_a[i], nCenarios);            
 Mb = rand(Dist_b[i], nCenarios);
 M[i,:] = Ma;
 N[i,:] = Mb;
end

Ce = 1:nCenarios;
Prob_Cen = (1/nCenarios)*ones(nCenarios);



a = [maximum(W[i,:]) for i in 1:24];

b = [minimum(W[i,:]) for i in 1:24];

c = [maximum(P[i,:]) for i in 1:24];

d = [minimum(P[i,:]) for i in 1:24];

# Wind farm model
gmax = 56400; #Maximum installed power of the wind farm [MW].
gmin = 0;
α = 0.95;
θ = -1000;
modelo=Model(SCIP.Optimizer);
@variable(modelo,Xi[i in tempo]>=0) # o que gostaria de vender em cada periodo
@variable(modelo,g[i in tempo, j in Ce]>=0) #geracao
@variable(modelo,yit[i in tempo, j in Ce]>=0) # g-x
#Risk variables
#@variable(modelo,Cvar)
#@variable(modelo,β)
@variable(modelo,R[j in Ce])
#@variable(modelo,γ[j in Ce]>=0)
#general restrictions
@constraint(modelo,[i in tempo,j in Ce],yit[i,j] >= g[i,j] - Xi[i])
@constraint(modelo,[i in tempo,j in Ce],yit[i,j] >= - g[i,j]+ Xi[i])
@constraint(modelo,[i in tempo, j in Ce],g[i,j]<= M[i,j]) 
@constraint(modelo,[i in tempo],Xi[i]<= gmax) #gmax é a cap instalada maxima
#risk constrainsts
#@constraint(modelo,REST,Cvar >= - θ) 
#@constraint(modelo,REST1[j in Ce],γ[j]>= β - R[j]) 
#@constraint(modelo,REST2,Cvar == β - sum(γ[j]/length(Ce) for j in Ce)/(1- α)) 
@constraint(modelo,REST3[j in Ce], R[j] == sum(N[i,j]*g[i,j] - w*N[i,j]*(yit[i,j]) for i in tempo))
#F.O
@objective(modelo,Max,sum(Prob_Cen[j]*R[j] for j in Ce))
optimize!(modelo)
#println(modelo)
#println(raw_status(modelo))

println(objective_value(modelo))

x_wfm = JuMP.value.(Xi);

feasible solution found by trivial heuristic after 0.2 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 24024 del conss, 0 add conss, 24024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       1000 del vars, 25024 del conss, 0 add conss, 25024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       1000 del vars, 25024 del conss, 0 add conss, 73048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       1000 del vars, 25024 del conss, 0 add conss, 97048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (5 rounds: 5 fast, 1 medium, 1 exhaustive):
 47023 deleted vars, 71024 deleted constraints, 0 added constraints, 97048 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 2001 variables (0 bin, 0 int, 0 impl, 2001 cont) and 2000 constraints
   2000 

## Pumped-storage plant model

In [3]:
# Pumped-storage plant:
# Criando variaveis para este modelo

α = 0.95
θ = -1000
gmax = 10 #Maximum installed power of the Pumped-storage plant [MW].
gmin = 0
modelo=Model(SCIP.Optimizer)

@variable(modelo,Xi[i in tempo]>=0) # o que gostaria de vender em cada periodo
@variable(modelo,g[i in tempo, j in Ce]>=0) # geracao
@variable(modelo,yit[i in tempo, j in Ce]>=0)
@variable(modelo,d[i in tempo, j in Ce]>=0) # Pumping power input of the pumped-storage plant in scenario in period [MW].
@variable(modelo,up[i in tempo, j in Ce]>=0) # Energy stored in the upper reservoir in scenario   at the end of period   [MWh].
@variable(modelo,ul[i in tempo, j in Ce]>=0) #Energy stored in the lower reservoir in scenario   at the end of period   [MWh].
@variable(modelo,ush[i in tempo, j in Ce]>=0, Int) # Integer variable that indicates the number of units that are 
# running in the pumping mode in scenario   in period     
@variable(modelo,tsh[i in tempo, j in Ce], Bin)


#variaveis ligadas a risco
#@variable(modelo,Cvar)
#@variable(modelo,β)
@variable(modelo,R[j in Ce]>=0)
#@variable(modelo,γ[j in Ce]>=0)

n = 0.99 # plotar grafico de lucro com n - eficiencias diferenes

upmax = 30
upmin = 0

ulmax = 30
ulmin = 0

ulf = 15 #Initial and final levels in the lower reservoir [MWh].
upf = 15 #Initial and final levels in the upper reservoir [MWh].
Dmax = 10
Dmin = 4

@constraint(modelo,[i in tempo,j in Ce],yit[i,j] >= g[i,j] - Xi[i]- d[i,j])
@constraint(modelo,[i in tempo,j in Ce],yit[i,j] >= - g[i,j]+ Xi[i] + d[i,j])
@constraint(modelo,[i in tempo[2:end],j in Ce],up[i,j] == up[i-1,j] + n*d[i,j] - g[i,j])
@constraint(modelo,[i in tempo[2:end],j in Ce],ul[i,j] == ul[i-1,j] - n*d[i,j] + g[i,j])
@constraint(modelo,[i in tempo,j in Ce],ulmin <= ul[i,j] <= ulmax)
@constraint(modelo,[i in tempo,j in Ce],upmin <= up[i,j] <= upmax)
@constraint(modelo,[j in Ce],upf == up[24,j])
@constraint(modelo,[j in Ce],ulf == ul[24,j])

@constraint(modelo,[j in Ce],up[1,j] == upf + n*d[1,j] - g[1,j])
@constraint(modelo,[j in Ce],ul[1,j] == ulf + g[1,j]- n*d[1,j])

@constraint(modelo,[i in tempo,j in Ce],Dmin <= d[i,j] <= Dmax)
@constraint(modelo,[i in tempo,j in Ce],g[i,j] <= gmax*tsh[i,j])
@constraint(modelo,[i in tempo,j in Ce],gmin*tsh[i,j] <= g[i,j]) 
@constraint(modelo,[i in tempo,j in Ce],- Dmax <= Xi[i] <= gmax)

#Restricoes ligadas ao risco
#@constraint(modelo,REST,Cvar >= - θ) 
#@constraint(modelo,REST1[j in Ce],γ[j]>= β - R[j]) 
#@constraint(modelo,REST2,Cvar == β - sum(γ[j]/length(Ce) for j in Ce)/(1- α)) 
@constraint(modelo,REST3[j in Ce], R[j] == sum(N[i,j]*(g[i,j] - d[i,j]) - w*N[i,j]*(yit[i,j]) for i in tempo))  

@objective(modelo,Max,sum(Prob_Cen[j]* R[j] for j in Ce))  
    
optimize!(modelo)
#println(raw_status(modelo))
println(objective_value(modelo))

x_psp = JuMP.value.(Xi);

presolving:
(round 1, fast)       50000 del vars, 146000 del conss, 0 add conss, 122024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       55000 del vars, 150000 del conss, 0 add conss, 155024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       55000 del vars, 150000 del conss, 0 add conss, 159024 chg bounds, 1000 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       55000 del vars, 150000 del conss, 0 add conss, 183048 chg bounds, 1000 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       55000 del vars, 150000 del conss, 0 add conss, 207048 chg bounds, 1000 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 1 medium, 1 exhaustive):
 55000 deleted vars, 150000 deleted constraints, 0 added constraints, 207048 tightened bounds, 0 added holes, 1000 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem h

## Mixed Model

In [4]:
# joint operation. 

# Efficiency of the pump-turbine cycle [p.u.]. Testar de .7 a 1 - Plotar grafico!

#eficiencia
n = 0.99
#parametros do cvar
α = 0.95
θ = -1000

#parametros da pumped storage
upmax = 30 
upmin = 0

ulmax = 30
ulmin = 0

ulf = 15 #Initial and final levels in the lower reservoir [MWh].
upf = 15 #Initial and final levels in the upper reservoir [MWh].
Dmax = 10
Dmin = 4

#parametros para wind e pumped storage:
gwmin = 0
gwmax = 56400
gbmin = 0
gbmax = 10

modelo=Model(SCIP.Optimizer)

@variable(modelo,Xi[i in tempo]>=0) # o que gostaria de vender em cada periodo
@variable(modelo,gw[i in tempo, j in Ce]>=0)
@variable(modelo,gb[i in tempo, j in Ce]>=0)
@variable(modelo,yit[i in tempo, j in Ce]>=0)
@variable(modelo,d[i in tempo, j in Ce]>=0) # Pumping power input of the pumped-storage plant in scenario in period [MW].
@variable(modelo,up[i in tempo, j in Ce])
@variable(modelo,ul[i in tempo, j in Ce])
#@variable(modelo,ush[i in tempo, j in Ce]>=0, Int) # Integer variable that indicates the number of units that are 
# running in the pumping mode in scenario   in period  OJO not being used   
@variable(modelo,tsh[i in tempo, j in Ce], Bin)

#variaveis ligadas a risco
#@variable(modelo,Cvar)
#@variable(modelo,β)
@variable(modelo,R[j in Ce]>=0)
#@variable(modelo,γ[j in Ce]>=0)


@constraint(modelo,[i in tempo,j in Ce],yit[i,j] >= gw[i,j] + gb[i,j] - Xi[i]- d[i,j])
@constraint(modelo,[i in tempo,j in Ce],yit[i,j] >= - gw[i,j] - gb[i,j]+ Xi[i] + d[i,j])
@constraint(modelo,[i in tempo[2:end],j in Ce],up[i,j] == up[i-1,j] + n*d[i,j] - gb[i,j])
@constraint(modelo,[i in tempo[2:end],j in Ce],ul[i,j] == ul[i-1,j] - n*d[i,j] + gb[i,j])
@constraint(modelo,[i in tempo,j in Ce],ulmin <= ul[i,j] <= ulmax)
@constraint(modelo,[i in tempo,j in Ce],upmin <= up[i,j] <= upmax)
@constraint(modelo,[j in Ce],upf == up[24,j])
@constraint(modelo,[j in Ce],ulf == ul[24,j])

@constraint(modelo,[j in Ce],up[1,j] ==  upf + n*d[1,j] - gb[1,j]) 
@constraint(modelo,[j in Ce],ul[1,j] ==  ulf + gb[1,j]- n*d[1,j]) 

@constraint(modelo,[i in tempo,j in Ce],Dmin <= d[i,j] <= Dmax)
@constraint(modelo,[i in tempo,j in Ce],gw[i,j] <= gwmax*tsh[i,j])
@constraint(modelo,[i in tempo,j in Ce],gwmin*tsh[i,j] <= gw[i,j]) 
@constraint(modelo,[i in tempo,j in Ce],gb[i,j] <= gbmax*tsh[i,j])
@constraint(modelo,[i in tempo,j in Ce],gbmin*tsh[i,j] <= gb[i,j]) 
@constraint(modelo,[i in tempo,j in Ce],- Dmax <= Xi[i] <= gbmax + gwmax)



@constraint(modelo,[i in tempo, j in Ce],gw[i,j]<= M[i,j]) 
@constraint(modelo,[i in tempo],Xi[i]<= gwmax) #gmax é a cap instalada maxima

#Restricoes ligadas ao risco
#@constraint(modelo,REST,Cvar >= - θ) 
#@constraint(modelo,REST1[j in Ce],γ[j]>= β - R[j]) 
#@constraint(modelo,REST2,Cvar == β - sum(γ[j]/length(Ce) for j in Ce)/(1- α)) 
@constraint(modelo,REST3[j in Ce], R[j] == sum(N[i,j]*(gw[i,j]+ gb[i,j] - d[i,j]) - w*N[i,j]*(yit[i,j]) for i in tempo))  

@objective(modelo,Max,sum(Prob_Cen[j]* R[j] for j in Ce))  

optimize!(modelo)
#println(raw_status(modelo))

println(objective_value(modelo))

x_mm = objective_value(modelo)

#value(Cvar)
#println(modelo)
#JuMP.value.(Xi)

presolving:
(round 1, fast)       26000 del vars, 218024 del conss, 0 add conss, 216024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       31000 del vars, 222024 del conss, 0 add conss, 249162 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       31000 del vars, 222024 del conss, 0 add conss, 253164 chg bounds, 1000 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       31000 del vars, 222024 del conss, 0 add conss, 277019 chg bounds, 1000 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       31000 del vars, 222024 del conss, 0 add conss, 301019 chg bounds, 1000 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 1 medium, 1 exhaustive):
 31000 deleted vars, 222024 deleted constraints, 0 added constraints, 301019 tightened bounds, 0 added holes, 1000 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem h

145063.95336703715

# Sample Average Approximation

## Wind Farm Model

In [5]:
# model definition
@stochastic_model wind_farm begin
    @stage 1 begin
        @parameters tempo = [ i for i in 1:24 ]
        @decision( wind_farm, Xi[i in tempo] >= 0)
        
        @objective( wind_farm, Max, 0 )
    end
    @stage 2 begin
        @parameters begin
            gmax = 56400
            gmin = 0
            w = 1
        end
        @uncertain ɱ[ i in tempo, jj in 1:2 ]
        
        @recourse( wind_farm, g[i in tempo] >= 0 )
        @recourse( wind_farm, yit[i in tempo] >= 0 )        
        @recourse( wind_farm, R )
        
        @objective( wind_farm, Max, R )
        
        @constraint( wind_farm, c1[i in tempo], yit[i] >= g[i] - Xi[i] )
        @constraint( wind_farm, c2[i in tempo], yit[i] >= -g[i] + Xi[i] )
        @constraint( wind_farm, c3[i in tempo], g[i] <= ɱ[i,1] )
        @constraint( wind_farm, c4[i in tempo], Xi[i] <= gmax )
        @constraint( wind_farm, Risk, R == sum( ɱ[i,2]*g[i] - w*ɱ[i,2]*yit[i] for i in tempo) )
    end
end      
            

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [6]:
#sampler definiton (ALL models use same sampler)
@sampler ParSampTime2 = begin
    a::Array{Float64,1}
    b::Array{Float64,1}
    c::Array{Float64,1}
    d::Array{Float64,1}
    tempo::Array{Int64,1}
    
    ParSampTime2(a,b,c,d,tempo) = new(a,b,c,d,tempo)
    
    @sample Scenario begin
        @parameters a b c d
        return @scenario ɱ[ i in tempo, jj in 1:2 ] =  [rand.(Uniform.(b,a))  rand.(Uniform.(d,c))] 
    end
end


a = [ maximum( W[i,:] ) for i in 1:24 ];
b = [ minimum( W[i,:] ) for i in 1:24 ];
c = [ maximum( P[i,:] ) for i in 1:24 ];
d = [ minimum( P[i,:] ) for i in 1:24 ];

sampler = ParSampTime2( a, b, c, d, tempo)


Scenario sampler

In [7]:
#run usual approach
sampled = instantiate( wind_farm, sampler, 1000, optimizer=SCIP.Optimizer )
optimize!( sampled )
x_wfm = optimal_decision( sampled )
print( objective_value( sampled ) )

feasible solution found by trivial heuristic after 0.2 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       1000 del vars, 49000 del conss, 0 add conss, 25024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       1000 del vars, 49000 del conss, 0 add conss, 73048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       1000 del vars, 49000 del conss, 0 add conss, 97048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 47023 deleted vars, 95000 deleted constraints, 0 added constraints, 97048 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 2001 variables (0 bin, 0 int, 0 impl, 2001 cont) and 2000 constraints
   2000 constraints of type <linear>
Presolving Time: 3.36

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|conf

In [8]:
#set optimizer for SAA
set_optimizer( wind_farm, SAA.Optimizer )
set_optimizer_attribute( wind_farm, InstanceOptimizer(), SCIP.Optimizer )
set_optimizer_attribute( wind_farm, Confidence(), 0.9 )
set_optimizer_attribute( wind_farm, NumSamples(), 100 )
set_optimizer_attribute( wind_farm, NumEvalSamples(), 300 )
load_model!( wind_farm.optimizer.optimizer, wind_farm,sampler, x_wfm )
wind_farm.optimizer.optimizer.algorithm

StochasticPrograms.SAA.SampleAverageApproximation{Float64,Array{Float64,1},StochasticModel{2,Tuple{StageParameters{NamedTuple{(:tempo,),Tuple{Array{Int64,1}}}},StageParameters{NamedTuple{(:gmax, :gmin, :w),Tuple{Int64,Int64,Int64}}}}},ParSampTime2}(Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)
, Scenario sampler, MathOptInterface.OptimizerWithAttributes(SCIP.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[]), [26.138917154569942, 25.361781646355848, 24.638180933282257, 24.907052562498496, 23.954870021093697, 23.816175863506967, 25.99965387060523, 25.436728916137294, 28.528476739910257, 28.103964194178317  …  26.843090115937866, 24.812823869865973, 25.656306487671706, 24.083291889088468, 24.15057834073436, 24.239566935922525, 23.22657147750145, 24.130912864945916, 23.80889448115173, 25.415152023876388], ConfidenceInterval{Float64}[], StochasticPrograms.SAA.SAAData{Float64}
  sample_size: Int64 16

In [9]:
#run SAA
MOI.optimize!( wind_farm.optimizer.optimizer )

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/gabriel/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
SAA gap Time: 0:00:35 (3 iterations)
  Confidence interval:  Confidence interval (p = 90%): [97170.68 − 97191.23]
  Relative error:       0.00021152757401269717
  Sample size:          32


In [10]:
#optimal decision from SAA
sp = optimal_instance( wind_farm.optimizer.optimizer )
print( objective_value(sp) )
optimal_decision( sp )

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       32 del vars, 1568 del conss, 0 add conss, 824 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       32 del vars, 1568 del conss, 0 add conss, 1656 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       32 del vars, 1568 del conss, 0 add conss, 2380 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       32 del vars, 1568 del conss, 0 add conss, 3148 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (5 rounds: 5 fast, 1 medium, 1 exhaustive):
 1527 deleted vars, 3040 deleted constraints, 0 added constraints, 3148 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 65 variables (0 bin, 0 int, 0 impl, 65 cont) and 64 constraints
     64 constraints of type <li

24-element Array{Float64,1}:
 21.890234194047625
 23.353650814583062
 21.10419632040725
 20.84287641055867
 17.286243733028908
 24.441736509298206
 18.519714802529137
 18.613288374941725
 23.380791047938974
 25.949391295906068
 27.271893998452974
 21.3054965086904
 31.770621727714715
 26.925042008210664
 27.277745117926127
 21.593587930182505
 29.68742257434126
 25.999014719554534
 22.192607359718334
 17.67517821926452
 26.785572087789657
 29.93030519828625
 20.633402738431467
 23.64116321826765

## Pumped-storage plant model

In [11]:
#model definition
@stochastic_model pump_storage_plant begin
    @stage 1 begin
        @parameters tempo = [ i for i in 1:24 ]
        @decision( pump_storage_plant, Xi[i in tempo] >= 0 )
        
        @objective( pump_storage_plant, Max, 0 )
    end
    @stage 2 begin
        @parameters begin
            gmax = 10
            gmin = 0
            w = 1
            n = 0.99
            upmax = 30
            ulmin = 0
            ulf = 15
            upf = 15
            Dmax = 10
            Dmin = 4
        end
        @uncertain ɱ[ i in tempo, jj in 1:2 ]
        
        @recourse( pump_storage_plant, g[i in tempo] >= 0 )
        @recourse( pump_storage_plant, yit[i in tempo] >= 0 )
        @recourse( pump_storage_plant, d[i in tempo] >= 0 ) 
        @recourse( pump_storage_plant, up[i in tempo] >= 0 ) 
        @recourse( pump_storage_plant, ul[i in tempo] >= 0 ) 
        @recourse( pump_storage_plant, tsh[i in tempo], Bin ) 
        @recourse( pump_storage_plant, R )
        
        @objective( pump_storage_plant, Max, R )
        
        @constraint( pump_storage_plant, c1[i in tempo], yit[i] >= g[i] - Xi[i] - d[i] )
        @constraint( pump_storage_plant, c2[i in tempo], yit[i] >= -g[i] + Xi[i] + d[i] )
        
        @constraint( pump_storage_plant, c3[i in tempo[2:end]], up[i] == up[i-1] + n*d[i] - g[i] )
        @constraint( pump_storage_plant, c4[i in tempo[2:end]], ul[i] == ul[i-1] - n*d[i] + g[i] )
        @constraint( pump_storage_plant, c5, up[1] == upf + n*d[1] - g[1] )
        @constraint( pump_storage_plant, c6, ul[1] == ulf + g[1]- n*d[1] )
        @constraint( pump_storage_plant, c7, up[24] == upf )
        @constraint( pump_storage_plant, c8, ul[24] == ulf )  
        @constraint( pump_storage_plant, c9[i in tempo], ulmin <= ul[i] <= ulmax )
        @constraint( pump_storage_plant, c10[i in tempo], upmin <= up[i] <= upmax )
 
        @constraint( pump_storage_plant, c11[i in tempo], Dmin <= d[i] <= Dmax )
        
        @constraint( pump_storage_plant, c12[i in tempo], g[i] <= gmax*tsh[i] )
        @constraint( pump_storage_plant, c13[i in tempo], gmin*tsh[i] <= g[i] )
        
        @constraint( pump_storage_plant, c14[i in tempo], - Dmax <= Xi[i] <= gmax )
        
        @constraint( pump_storage_plant, Risk, R == sum( ɱ[i,2]*(g[i] - d[i]) - w*ɱ[i,2]*yit[i] for i in tempo) )
    end
end   

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [12]:
#run usual approach
sampled = instantiate( pump_storage_plant, sampler, 1000, optimizer=SCIP.Optimizer )
optimize!( sampled )
x_psp = optimal_decision( sampled )
print( objective_value( sampled ) )

presolving:
(round 1, fast)       26000 del vars, 146000 del conss, 0 add conss, 122024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       31000 del vars, 151000 del conss, 0 add conss, 131029 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       31000 del vars, 151000 del conss, 0 add conss, 135024 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       31000 del vars, 151000 del conss, 0 add conss, 159048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       31000 del vars, 151000 del conss, 0 add conss, 183048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 1 medium, 1 exhaustive):
 31000 deleted vars, 151000 deleted constraints, 0 added constraints, 183048 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 114024 va

In [13]:
#set optimizer for SAA
set_optimizer( pump_storage_plant, SAA.Optimizer )
set_optimizer_attribute( pump_storage_plant, InstanceOptimizer(), SCIP.Optimizer )
set_optimizer_attribute( pump_storage_plant, Confidence(), 0.9 )
set_optimizer_attribute( pump_storage_plant, NumSamples(), 100 )
set_optimizer_attribute( pump_storage_plant, NumEvalSamples(), 300 )
load_model!( pump_storage_plant.optimizer.optimizer, pump_storage_plant, sampler, x_psp )
pump_storage_plant.optimizer.optimizer.algorithm

StochasticPrograms.SAA.SampleAverageApproximation{Float64,Array{Float64,1},StochasticModel{2,Tuple{StageParameters{NamedTuple{(:tempo,),Tuple{Array{Int64,1}}}},StageParameters{NamedTuple{(:gmax, :gmin, :w, :n, :upmax, :ulmin, :ulf, :upf, :Dmax, :Dmin),Tuple{Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64}}}}},ParSampTime2}(Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)
, Scenario sampler, MathOptInterface.OptimizerWithAttributes(SCIP.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[]), [6.0, 0.0, 6.0, 0.0, 2.8000000000000007, 0.0, 0.0, 0.0, 0.0, 0.0  …  3.3800000000000026, 2.1999999999999957, 0.0, 0.0, 5.050000000000003, 5.310000000000003, 0.0, 6.0, 3.4899999999999993, 0.0], ConfidenceInterval{Float64}[], StochasticPrograms.SAA.SAAData{Float64}
  sample_size: Int64 16
  interval: ConfidenceInterval{Float64}
, StochasticPrograms.SAA.SAAParameters{Float64}
  confidence: Float64 0.9
  n

In [14]:
#run SAA
MOI.optimize!( pump_storage_plant.optimizer.optimizer )

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/gabriel/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
SAA gap Time: 0:05:52 (5 iterations)
  Confidence interval:  Confidence interval (p = 90%): [6103.09 − 6158.17]
  Relative error:       0.008982817978663212
  Sample size:          128


In [15]:
#optimal decision from SAA
sp = optimal_instance( pump_storage_plant.optimizer.optimizer )
print( objective_value(sp) )
optimal_decision( sp )

presolving:
(round 1, fast)       3328 del vars, 18688 del conss, 0 add conss, 15640 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       3968 del vars, 19328 del conss, 0 add conss, 16794 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       3968 del vars, 19328 del conss, 0 add conss, 17304 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       3968 del vars, 19328 del conss, 0 add conss, 20400 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       3968 del vars, 19328 del conss, 0 add conss, 23472 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 1 medium, 1 exhaustive):
 3968 deleted vars, 19328 deleted constraints, 0 added constraints, 23472 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 14616 variables (0 bin, 0 i

24-element Array{Float64,1}:
 6.0
 0.0
 2.8000000000000043
 0.0
 6.0
 0.0
 2.460000000000001
 0.0
 0.0
 3.780000000000001
 6.0
 5.920000000000002
 0.0
 6.0
 0.0
 0.0
 0.0
 3.6999999999999993
 1.8599999999999994
 6.0
 0.0
 2.1800000000000033
 0.0
 4.720000000000001

## Mixed Model

In [16]:
#model definition
@stochastic_model mixed_model begin
    @stage 1 begin
        @parameters tempo = [ i for i in 1:24 ]
        @decision( mixed_model, Xi[i in tempo] >= 0 )
        
        @objective( mixed_model, Max, 0 )
    end
    @stage 2 begin
        @parameters begin
            gmax = 10
            gmin = 0
            w = 1
            n = 0.99
            upmax = 30
            ulmin = 0
            ulf = 15
            upf = 15
            Dmax = 10
            Dmin = 4
            gwmin = 0
            gwmax = 56400
            gbmin = 0
            gbmax = 10
        end
        @uncertain ɱ[ i in tempo, jj in 1:2 ]
        
        @recourse( mixed_model, gw[i in tempo] >= 0 )
        @recourse( mixed_model, gb[i in tempo] >= 0 )
        @recourse( mixed_model, yit[i in tempo] >= 0 )
        @recourse( mixed_model, d[i in tempo] >= 0 ) 
        @recourse( mixed_model, up[i in tempo] ) 
        @recourse( mixed_model, ul[i in tempo] ) 
        @recourse( mixed_model, tsh[i in tempo], Bin ) 
        @recourse( mixed_model, R )
        
        @objective( mixed_model, Max, R )
        
        @constraint( mixed_model, c1[i in tempo], yit[i] >= gw[i] + gb[i] - Xi[i] - d[i] )
        @constraint( mixed_model, c2[i in tempo], yit[i] >= - gw[i] - gb[i] + Xi[i] + d[i] )
        
        @constraint( mixed_model, c3[i in tempo[2:end]], up[i] == up[i-1] + n*d[i] - gb[i] )
        @constraint( mixed_model, c4[i in tempo[2:end]], ul[i] == ul[i-1] - n*d[i] + gb[i] )
        @constraint( mixed_model, c5, up[1] == upf + n*d[1] - gb[1] )
        @constraint( mixed_model, c6, ul[1] == ulf + gb[1]- n*d[1] )
        @constraint( mixed_model, c7, up[24] == upf )
        @constraint( mixed_model, c8, ul[24] == ulf )  
        @constraint( mixed_model, c9[i in tempo], ulmin <= ul[i] <= ulmax )
        @constraint( mixed_model, c10[i in tempo], upmin <= up[i] <= upmax )
 
        @constraint( mixed_model, c11[i in tempo], Dmin <= d[i] <= Dmax )
        
        @constraint( mixed_model, c12[i in tempo], gw[i] <= gwmax*tsh[i] )
        @constraint( mixed_model, c13[i in tempo], gwmin*tsh[i] <= gw[i] )
        @constraint( mixed_model, c14[i in tempo], gb[i] <= gbmax*tsh[i] )
        @constraint( mixed_model, c15[i in tempo], gbmin*tsh[i] <= gb[i] )        
        @constraint( mixed_model, c16[i in tempo], gw[i]<= ɱ[i,1]) 
        
        @constraint( mixed_model, c17[i in tempo], Xi[i]<= gwmax ) #gmax é a cap instalada maxima
        @constraint( mixed_model, c18[i in tempo], - Dmax <= Xi[i] <= gbmax + gwmax )
        
        @constraint( mixed_model, Risk, R == sum( ɱ[i,2]*(gw[i] + gb[i] - d[i]) - w*ɱ[i,2]*yit[i] for i in tempo) )
    end
end  

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [17]:
#run usual approach
sampled = instantiate( mixed_model, sampler, 1000, optimizer=SCIP.Optimizer )
optimize!( sampled )
x_mm = optimal_decision( sampled )
print( objective_value( sampled ) )

presolving:
(round 1, fast)       26000 del vars, 242000 del conss, 0 add conss, 216048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       31000 del vars, 247000 del conss, 0 add conss, 225053 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       31000 del vars, 247000 del conss, 0 add conss, 229048 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       31000 del vars, 247000 del conss, 0 add conss, 253072 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       31000 del vars, 247000 del conss, 0 add conss, 277072 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 1 medium, 1 exhaustive):
 31000 deleted vars, 247000 deleted constraints, 0 added constraints, 277072 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 138024 va

In [18]:
#set optimizer for SAA
set_optimizer( mixed_model, SAA.Optimizer )
set_optimizer_attribute( mixed_model, InstanceOptimizer(), SCIP.Optimizer )
set_optimizer_attribute( mixed_model, Confidence(), 0.9 )
set_optimizer_attribute( mixed_model, NumSamples(), 100 )
set_optimizer_attribute( mixed_model, NumEvalSamples(), 300 )
load_model!( mixed_model.optimizer.optimizer, mixed_model, sampler, x_mm )
mixed_model.optimizer.optimizer.algorithm

StochasticPrograms.SAA.SampleAverageApproximation{Float64,Array{Float64,1},StochasticModel{2,Tuple{StageParameters{NamedTuple{(:tempo,),Tuple{Array{Int64,1}}}},StageParameters{NamedTuple{(:gmax, :gmin, :w, :n, :upmax, :ulmin, :ulf, :upf, :Dmax, :Dmin, :gwmin, :gwmax, :gbmin, :gbmax),Tuple{Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64}}}}},ParSampTime2}(Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)
, Scenario sampler, MathOptInterface.OptimizerWithAttributes(SCIP.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[]), [28.30182502734921, 27.348000406627396, 28.27590055648251, 26.93109339614333, 27.024092120313536, 26.26578955215617, 25.924664202434258, 30.41174970921655, 31.877647269999628, 30.98052670730278  …  30.71237687076109, 28.943553329932563, 30.648199764950675, 27.379284220056206, 25.75969354653892, 27.46921099752493, 25.290112387926474, 26.89844739804

In [19]:
#run SAA
MOI.optimize!( mixed_model.optimizer.optimizer )

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/gabriel/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
SAA gap Time: 0:06:17 (4 iterations)
  Confidence interval:  Confidence interval (p = 90%): [141988.19 − 142760.92]
  Relative error:       0.005427493099451191
  Sample size:          64


In [20]:
#optimal decision from SAA
sp = optimal_instance( mixed_model.optimizer.optimizer )
print( objective_value(sp) )
optimal_decision( sp )

presolving:
(round 1, fast)       1664 del vars, 15488 del conss, 0 add conss, 13872 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       1984 del vars, 15808 del conss, 0 add conss, 14453 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       1984 del vars, 15808 del conss, 0 add conss, 14704 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       1984 del vars, 15808 del conss, 0 add conss, 16264 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       1984 del vars, 15808 del conss, 0 add conss, 17800 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 1 medium, 1 exhaustive):
 1984 deleted vars, 15808 deleted constraints, 0 added constraints, 17800 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 8856 variables (0 bin, 0 in

24-element Array{Float64,1}:
 28.39078421647983
 30.437835381405876
 22.558488423065697
 26.562370909323608
 29.057475513162572
 24.29944039269559
 27.070079330421798
 29.70739934854776
 23.733656316308323
 36.6891679429568
 33.44296525183057
 34.120517984007314
 27.711429031741755
 26.417706550992
 26.992880936665504
 36.731317512925784
 28.856698181193906
 26.405239513540046
 19.676843034350604
 29.519934193472835
 28.53878325446354
 28.01147755400264
 23.92892473459318
 27.51289316841018

## References
@article{spjl,
  title     = {Efficient Stochastic Programming in {J}ulia},
  author    = {Martin Biel and Mikael Johansson},
  journal   = {arXiv preprint arXiv:1909.10451},
  year      = {2019}
}

@article{DunningHuchetteLubin2017,
author = {Iain Dunning and Joey Huchette and Miles Lubin},
title = {JuMP: A Modeling Language for Mathematical Optimization},
journal = {SIAM Review},
volume = {59},
number = {2},
pages = {295-320},
year = {2017},
doi = {10.1137/15M1020575},
}
